<h1> NOTEBOOK TO CREATE DLC PROJECT, EXTRACT FRAMES & LABEL DATA </h1>

make sure you are running this notebook in properly setup conda enviroment. The nbkGPU enviroment set up on server 2 works properly. 

In [2]:
#Errors here mean there are issues with the enviroment dependencies or PATH configuration
import deeplabcut
import os
from random import sample
task = 'box1_cam2_ground'
experimenter = 'spencerloggia'

Get all videos from the provided video source directory

In [ ]:
BASE_DIR = r'F:\MysoreData\nbk\mouseVideoAnalysis\Box1\video_data\mitg12'
videodirs = os.listdir(BASE_DIR)
video = []
for i, videodir in enumerate(sample(videodirs, 25)):
    try:
        videos = os.listdir(os.path.join(BASE_DIR, videodir, 'ground'))
    except FileNotFoundError:
        print('skipping')
        continue
    videos = [path for path in videos if 'camera-2' in path]
    
    #print(videos)
    try:
        for vid in sample(videos, 1):
            new_vid_name = vid[0:-4] + '_num' + str(i) + '.avi'
            os.rename(os.path.join(BASE_DIR, videodir, 'ground', vid), os.path.join(BASE_DIR, videodir, 'ground', new_vid_name))
            video.append(os.path.join(BASE_DIR, videodir, 'ground', new_vid_name))  
    except ValueError:
        print("skipping " + str(videodir))
print(video)
num_videos = len(video)

Generate the deeplabcut project, change working directory or else it is created in current directory. The output is the path to the config file. 

In [ ]:
path_config_file=deeplabcut.create_new_project(task,experimenter,video, working_directory='F:\\MysoreData\\nbk\\mouseVideoAnalysis\\Box1\\cam2\\', copy_videos=True)
print(path_config_file)

At this point take a moment to edit the config.yaml file in the new project directory. Set the num2pick variable can be set to change number labeled frames per video. Also edit batch_size to change number of labelled images in each set. <br> 

<h1> Now we can extract frames!</h1><br>
<p>This shows you one frame in the DLC gui from each video in your videos array. It asks you to crop the frame to the area you care about. DLC does not recommend croping to an images size greater than 600X800px <br>
    Deep lab cut also now selects a number of frames equal to the num2pick configuiration from each video for labelling. These are subdevided into both the training and test sets.<p/>
    
Make sure that crop parameters for ground state match the mouse tracking network
    
                                   

In [ ]:
#load matplotlib so outputs can be displayed 
%matplotlib inline 
path_config_file = r'F:\MysoreData\nbk\mouseVideoAnalysis\Box1\cam2\box1_cam2_ground-spencerloggia-2021-02-14\config.yaml'
deeplabcut.extract_frames(path_config_file,'automatic','uniform', userfeedback=False, crop=True)

### <h1> Now we label the test and training data! </h1> 
    
Once the labelling gui opens, you may have to expand the bottom menu to make sure all buttons are visable.
There will be a folder with the extracted frames for each video. Go into each and right click to place the current label. Skip a label by choosing the next one on the right hand menu.

For consistency: BP1: left ear, BP2: right ear, BP3: nose (behind nostrils)

In [3]:
%gui wx
%matplotlib inline 
path_config_file = r'F:\MysoreData\nbk\mouseVideoAnalysis\Box1\cam2\box1_cam2_ground-spencerloggia-2021-02-14\config.yaml'
deeplabcut.label_frames(path_config_file)

KeyError: 'labeled-data\\camera-2_ground_state_num3_num6\\img02.png'

KeyError: 'labeled-data\\camera-2_ground_state_num6_num20\\img12.png'

Found new frames..
You can now check the labels, using 'check_labels' before proceeding. Then, you can use the function 'create_training_dataset' to create the training dataset.


In [ ]:
# create collected data 
data = ""
path = 'F:\\MysoreData\\nbk\\mouseVideoAnalysis\\Box1\\cam1\\box1_cam1-spencerloggia-2021-02-07\\labeled-data'
folders = os.listdir(path)
first = True 
for f in folders:
    if 'labeled' in f or '.' in f:
        continue
    file = open(os.path.join(path, f, 'CollectedData_spencerloggia.csv'), 'r')
    for i, line in enumerate(file):
        if not first:
            if i < 3:
                continue
        first = False
        data += line
    del f
file = open(os.path.join(path,'CollectedData_spencerloggia.csv'), 'w')
file.write(data)
import pandas as pd
df = pd.read_csv(os.path.join(path,'CollectedData_spencerloggia.csv'))
df.to_hdf(os.path.join(path,'CollectedData_spencerloggia.h5'), key='df_with_missing')
    

The last step is just an automatic check of the labelled data. You should see no errors or something has gone wrong. 

In [4]:
path_config_file = r'F:\MysoreData\nbk\mouseVideoAnalysis\Box1\cam2\box1_cam2_ground-spencerloggia-2021-02-14\config.yaml'

deeplabcut.check_labels(path_config_file) #this creates a subdirectory with the frames + your labels

  0%|                                                                                         | 0/2 [00:00<?, ?it/s]

Creating images with labels by spencerloggia.


100%|█████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  5.88it/s]

If all the labels are ok, then use the function 'create_training_dataset' to create the training dataset!


<h1> Next Steps: </h1>
<h2> In order to actually train the network, we must use another notebook that runs in a docker container. This allows tensorflow to communicate properly with GPUs in windows </h2>
run the below cell to get the path_config_file location, copy this path. Open the Docker_DLC_GPU notebook in this directory and set the path_config_file variable there to the same thing. 

In [ ]:
print(path_config_file)